In [ ]:
import pandas as pd

In [ ]:
x_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(x_train_path)

In [ ]:
X_train.head()

## Exploration

In [ ]:
len(X_train)

In [ ]:
X_train["piezo_measurement_date"].sort_values()

In [ ]:
X_train["piezo_station_bss_id"].unique()

In [ ]:
len(X_train[X_train["piezo_station_bss_id"] == "BSS001TTQQ"])

In [ ]:
print_full(X_train[X_train["piezo_station_bss_id"] == "BSS001TTQQ"])

## Dates sur X_test

In [ ]:
x_test_path = "data/X_test_Hi5.csv"
X_test = pd.read_csv(x_test_path)

In [ ]:
len(X_test)

In [ ]:
X_test["piezo_measurement_date"].sort_values()

In [ ]:
X_test["piezo_measurement_date"].sort_values().hist()

In [ ]:
X_train["piezo_station_bss_id"].unique()

## Other sheniningans

In [ ]:
X_train.rename(columns={"piezo_groundwater_level_category": "target"}, inplace=True)

In [ ]:
X_train["target"].unique()

In [ ]:
X_train[X_train["target"].isna()]

## Corrélation entre les features et la target

Est-ce que ça fait même sens de faire ça vu que la target est catégorielle ?

In [ ]:
from pandas import factorize
import numpy as np

In [ ]:
numerical_cols = X_train.select_dtypes(include=np.number)

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
def corrwith(df, label):
    df["target_labels"] = df["target"].apply(lambda x: 1 if x == label else 0)
    print_full(abs(numerical_cols.corrwith(df["target_labels"])).sort_values())

In [ ]:
corrwith(X_train, ["target"])